Background reading: https://www.oreilly.com/learning/handling-missing-data

inspired by https://www.dataquest.io/blog/programming-best-practices-for-data-science/

data source: https://www.lendingclub.com/info/download-data.action

In [ ]:
import time
import pandas
print(pandas.__version__)

In [ ]:
pandas.options.display.max_rows = 999

In [ ]:
start_time = time.time()
loans_2007 = pandas.read_csv('../data/LoanStats3a.csv', skiprows=1, low_memory=False)
print(time.time() - start_time,'seconds')
loans_2007.shape

In [ ]:
loans_2007.head(6).T

## Empty columns

are there columns that have no values?

In [ ]:
loans_2007.isnull().head()

In [ ]:
loans_2007.isnull().sum()

In [ ]:
loans_2007.shape[0]

In [ ]:
loans_2007.isnull().sum()==loans_2007.shape[0]

Drop columns that are not useful

In [ ]:
series_of_bool_indicating_whether_column_is_empty = loans_2007.isnull().sum()==loans_2007.shape[0]
loans_2007.shape

for col_name, bool_value in series_of_bool_indicating_whether_column_is_empty.iteritems():
    if bool_value:
        loans_2007.drop([col_name],axis=1,inplace=True)

In [ ]:
loans_2007.shape

The original size was (42538, 145), so we deleted 81 columns.


For each column, what is the ratio of rows with missing entries to total row count?

In [ ]:
len(loans_2007) # returns number of rows in dataframe

In [ ]:
loans_2007.isnull().sum()/len(loans_2007)

It would be better to view the series as a sorted sequence. 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.sort_values.html

I wasn't aware of the existance of ".sorted_values()", but I figured it was a common task and <a href="https://www.google.com/search?q=pandas+sort+series">Google'd for the concept</a>.

In [ ]:
(loans_2007.isnull().sum()/len(loans_2007)).sort_values(ascending=False)

There are many columns that have 0.0071% of the entries containing NaN. How many rows is that?

In [ ]:
loans_2007['total_rec_int'].isnull().sum()

The fact that many columns have the same number of NaNs is suspicious to me. To investigate, inspect those three rows

In [ ]:
loans_2007[loans_2007['total_rec_int'].isnull()]

To summarize what I'm seeing, for three rows there is text in one column.

Let's count how many NaNs are in the 'id' column:

In [ ]:
loans_2007['id'].isnull().sum()

In [ ]:
loans_2007.shape

The 'id' column is empty except for those three rows

My diagnosis is that the "id" column should not be part of the data analysis. 

The appropriate action would be to follow up with the person or team that supplied the data 

Before dropping the 'id' column and those three rows, let's validate that all other cells are NaN:

In [ ]:
loans_2007[loans_2007['total_rec_int'].isnull()].isnull().sum().sum()

In [ ]:
loans_2007[loans_2007['total_rec_int'].isnull()].shape

In [ ]:
3*64

Out of the 192 cells associated with the three rows, 189 cells are empty. That means we can safely drop both the "id" column and the three rows

In [ ]:
print('before:',loans_2007.shape)

loans_2007.drop(['id'],axis=1,inplace=True)
loans_2007.dropna(how='all',axis=0,inplace=True)

print('after:',loans_2007.shape)

# drop all rows and columns where entire row or column is NaN
If we wanted to be quick, we could have dropped all rows and columns containing NaNs

To show this, reload the data

In [ ]:
start_time = time.time()
loans_2007 = pandas.read_csv('../data/LoanStats3a.csv', skiprows=1, low_memory=False)
print(time.time() - start_time,'seconds')
loans_2007.shape

In [ ]:
print('before:',loans_2007.shape)

loans_2007.dropna(how='all',axis=0,inplace=True) # rows 
loans_2007.dropna(how='all',axis=1,inplace=True) # columns

print('after:',loans_2007.shape)

The value of not taking the quick approach is that we gain understanding of the input data

## Are there columns where all rows have the same value?

In [ ]:
for this_column in loans_2007.columns:
    if (loans_2007[this_column].nunique()==1):
        print(this_column)

These columns probably aren't going to yield much useful information

In [ ]:
print('before:',loans_2007.shape)

for this_column in loans_2007.columns:
    if (loans_2007[this_column].nunique()==1):
        loans_2007.drop([this_column],axis=1,inplace=True)
        
print('after:',loans_2007.shape)

We can inspect the remaining columns 

In [ ]:
def uniq_per_col(df,count):
    for this_column in df.columns:
        print("==== ",this_column,"has",df[this_column].nunique(),"unique entries ====")
        print(df[this_column].value_counts().head(count))

In [ ]:
uniq_per_col(loans_2007,5)